ปฏิบัติการครั้งที่ 5 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้น share แล้วส่ง link มาใน mango.cmu.ac.th

ดาวน์โหลดข้อมูลรถยนต์ชนิดต่างใน link ข้างล่างนี้
https://donlapark.pages.dev/229351/data/Auto.csv
หรือรันโค้ดข้างล่าง

In [1]:
!wget https://donlapark.pages.dev/229351/data/Auto.csv

--2025-08-08 12:27:27--  https://donlapark.pages.dev/229351/data/Auto.csv
Resolving donlapark.pages.dev (donlapark.pages.dev)... 172.66.44.200, 172.66.47.56, 2606:4700:310c::ac42:2f38, ...
Connecting to donlapark.pages.dev (donlapark.pages.dev)|172.66.44.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18079 (18K) [text/csv]
Saving to: ‘Auto.csv’

Auto.csv            100%[===================>]  17.66K  --.-KB/s    in 0s      

2025-08-08 12:27:28 (79.5 MB/s) - ‘Auto.csv’ saved [18079/18079]



In [2]:
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
# อ่านไฟล์ csv ก่อนเพื่อหา missing values
auto_df = pd.read_csv('Auto.csv',na_values=["?"])
predictor_names = auto_df.columns

# ลบแถวที่มี missing values
auto_df = auto_df.dropna()
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


### ตัวแปรต่าง ๆ มีดังนี้
mpg

    miles per gallon
cylinders

    Number of cylinders between 4 and 8
displacement

    Engine displacement (cu. inches)
horsepower

    Engine horsepower
weight

    Vehicle weight (lbs.)
acceleration

    Time to accelerate from 0 to 60 mph (sec.)
year

    Model year (modulo 100)
origin

    Origin of car (1. American, 2. European, 3. Japanese)
name

    Vehicle name


In [16]:
auto_df = auto_df.drop(['name', 'origin'],axis=1)

auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


## 1. Multivariate linear regression

เปลี่ยน $\texttt{dataframe}$ ให้เป็น numpy array ด้วย `dataframe.to_numpy()`

1. Response: $\texttt{y}$ เป็นคอลัมน์เวกเตอร์ของ `mpg`
2. Predictors: $\texttt{X}$ เป็นเมทริกซ์ของตัวแปรที่เหลือ (ใช้ `auto_df.drop`)


In [19]:

y = auto_df['mpg'].to_numpy()
X = auto_df.drop(['mpg'],axis=1).to_numpy()

คอลัมน์แรกของ $\texttt{X}$ ต้องเป็น $(1, 1, \ldots ,1 )^T$ (จำนวนเลข 1 เท่ากับจำนวนแถวของ $\texttt{X}$)  
สร้างเวกเตอร์ที่มีแต่เลข 1 และมีความยาวที่เหมาะสมด้วยคำสั่งใดคำสั่งหนึ่งต่อไปนี้  
1. $\texttt{[1]*X.shape[0]}$  
2. $\texttt{np.ones(X.shape[0])}$  

In [5]:
X.shape

(392, 8)

In [15]:
ones =  np.ones(X.shape[0]) # Use np.ones for a numpy array of ones

# ใส่ ones ให้เป็นคอลัมน์แรกของ X
X = np.c_[ones,X]

In [7]:
#เช็คคำตอบ
print(X)

[[1 8 307.0 ... 70 1 'chevrolet chevelle malibu']
 [1 8 350.0 ... 70 1 'buick skylark 320']
 [1 8 318.0 ... 70 1 'plymouth satellite']
 ...
 [1 4 135.0 ... 82 1 'dodge rampage']
 [1 4 120.0 ... 82 1 'ford ranger']
 [1 4 119.0 ... 82 1 'chevy s-10']]


### Exercise 1: จงเขียนฟังก์ชันต่อไปนี้
1. `linear_model` มี input เป็นแมทริกซ์ `X` เวกเตอร์ `y` และ output เป็นเวกเตอร์ของสัมประสิทธิ์
2. `predict` มี input เป็นแมทริกซ์ `X` เวกเตอร์ของสัมประสิทธิ์ `beta` และ output เป็นเวกเตอร์ของค่าทำนาย `y_pred`

1. Matrix multiplication $AB = \texttt{A@B}$
2. Transpose: $X^T = \texttt{X.T}$
3. Inverse: $X^{-1} = \texttt{np.linalg.inv(X)}$

$$ \hat{\beta} = (X^TX)^{-1}X^Ty $$

In [26]:
# Let's start by implementing linear regression from scratch
# using the formula provided in the class

def linear_model(X, y):
    """X: เมทริกซ์ของตัวแปรต้น"""
    """y: เวกเตอร์ของตัวแปรตาม"""
    """Return: เวกเตอร์ของ parameter beta จากการสร้าง linear regressions model
    ด้วย Ordinary Least Squares (OLS)"""
    # TODO: your code here
    beta = np.linalg.inv(X.T@X)@X.T@y

    return beta


def predict(X, beta):
    """beta: array ของสัมประสิทธิ์"""
    """X: เมทริกซ์ของตัวแปรต้น"""
    """Return: ค่าทำนาย y_pred = X*beta """
    # TODO: your code here
    y_pred = X@beta

    return y_pred

In [27]:
beta = linear_model(X, y)
y_pred = predict(X, beta)

print("Coefficients are:", beta)
print("Predictions are:", y_pred)

Coefficients are: [-0.5226089   0.01022108 -0.020873   -0.00639456 -0.05202195  0.61025869]
Predictions are: [15.93081361 14.45720405 16.11263762 15.93670419 16.10071197 10.51021782
 10.27543153 10.53128391  9.67525181 13.49634208 15.59946068 15.17857818
 14.95056695 18.23756934 23.85149163 20.70116218 21.04691637 22.47738545
 25.4075601  27.85849004 21.93938914 23.54965655 23.61026333 24.5700506
 22.02475307  7.48992449  8.73758029  8.68094775  6.39448743 26.01781879
 25.50668622 25.43458909 22.95714525 17.50355598 18.56684981 18.98997897
 18.64504732 11.74185816 10.43958016 12.2762232  12.39844199  7.02172716
  8.71466792  6.09084578 20.89073634 24.7795066  18.89340516 20.0843144
 25.76559234 26.24104628 26.30753902 26.59195154 28.28090927 29.28279009
 28.2609542  27.13912286 25.6470868  26.69569542 26.07663641 24.9880403
 26.2074288  11.93647037 11.52899822 12.73330182 13.0723569  15.65493241
  9.60277136 10.60920749 10.79899231 10.95329347 25.45996983 14.19610698
 13.24891771 11.63

#### จงหา $n$ (จำนวนแถวของ $\texttt{X}$) และ $p$ (จำนวนตัวแปร) ด้วย method $\texttt{X.shape[...]}$

In [28]:
n = X.shape[0]

p = X.shape[1]-1

print('Number of observations:', n)
print('Number of variables:', p)

Number of observations: 392
Number of variables: 5


## 2. F-statistic

### Exercise 2: เขียนสมมติฐานหลัก ($H_0$) และสมมติฐานแย้ง ($H_a$) ที่ใช้ F-statistic ในการทดสอบ



* $H_0$ : ไม่มีตัวแปรต้นใดเลยที่ส่งผลต่อตัวแปรตาม
หรือ
สัมประสิทธิ์ทั้งหมดเท่ากับ 0
*   $H_a$ : มีอย่างน้อยหนึ่งตัวแปรต้นที่มีอิทธิพลต่อตัวแปรตาม
หรือ
อย่างน้อยหนึ่งสัมประสิทธิ์ไม่เท่ากับ 0

### Exercise 3: จากเวกเตอร์ของ outcomes `y` คำนวณ Total Sum of Squares (TSS)

$$ \text{TSS} = \sum_{i=1}^n (y_i-\bar{y})^2 $$

$$ \text{RSS} = \sum_{i=1}^n (y_i-\hat{y})^2 $$
โดยใช้คำสั่งต่อไปนี้

$\texttt{np.sum}([y_1,y_2,...,y_n])=y_1+y_2+...+y_n$

$\texttt{np.square}([y_1,y_2,...,y_n])=[y_1^2,y^2_2,...,y^2_n]$

In [29]:
# Calculate Total Sum of Squares
y_mean = y.mean()

TSS = np.sum(np.square(y-y_mean))

RSS = np.sum(np.square(y-y_pred))

คำนวณ $F$-statistic ด้วย
$$F_{p,n-p-1} = \frac{(\text{TSS}-\text{RSS})/p}{\text{RSS}/(n-p-1)}$$
หา $n$ และจำนวนของตัวแปร $p$ ด้วย $\texttt{X.shape}[\cdot]$

### Exercise 4: คำนวณค่า F

In [30]:
numerator = (TSS - RSS) / p
denominator = RSS / (n - p - 1)
F = numerator / denominator
print(F)

317.97399948008183


In [31]:
# Compute p_value of the F-statistic

p_value = stats.f.sf(F, p, n-p-1)

print(p_value)

1.9865950228602433e-134


### Exercise 5: จากการทดสอบสมมติฐานที่นัยสำคัญ 0.05 เราได้ข้อสรุปอย่างไรจากค่า p-value ที่ได้ข้างบน

### คำตอบของ Exercise 5:
ที่ระดับนัยสำคัญ 0.05 เนื่องจากค่า p-value มากกว่า 0.05
เราจึงยอมรับสมมติฐานหลักได้




## 3. คำนวณ $t$-statistic

#### ขั้นตอนในการคำนวณ $t$-statistic

\begin{align*}
\text{RSE} &= \sqrt{\frac{\text{RSS}}{n-p-1}} \\
\text{SE}(\hat{\beta}) &=\text{RSE}\cdot\sqrt{\text{diag}\left( (X^TX)^{-1}\right)} \\
t &= \frac{\hat{\beta}}{\text{SE}(\hat{\beta})}
\end{align*}

In [42]:
def t_to_p_values(t_statistics):
    """t_statistics: numpy array ของค่า t-statistic ของตัวแปรต่างๆ
    Return: เวกเตอร์ของค่า p-value ของค่า t-statistic ที่ให้มา"""
    RSE = np.sqrt(RSS / (n - p - 1))
    SE = RSE * np.sqrt(np.diag(np.linalg.inv(X.T @ X)))
    return stats.t.sf(np.abs(t_statistics), n-p-1)*2

# Standard error of each variable in X
    p = t_to_p_values(t)


### Exercise 6: จงใช้ฟังก์ชัน `t_to_p_values` ในการหาว่ามีตัวแปรใดบ้างที่มีค่า p-value น้อยกว่า 0.05 **แล้วระบุตัวแปรทั้งหมดใน text block ข้างล่าง**  

#### ใช้คำสั่ง `A.diagonal()` เพื่อดึงค่าในแนวทแยงของ `A`

In [34]:
# TODO: your code here
RSE = np.sqrt(RSS / (n - p - 1))
SE = RSE * np.sqrt(np.diag(np.linalg.inv(X.T @ X)))
t = beta / SE


list ชื่อตัวแปร

In [35]:
print("List of predictors:", predictor_names[1:-2])

List of predictors: Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'year'],
      dtype='object')


### คำตอบของ Exercise 6:

ตัวแปรที่มีค่า p-value น้อยกว่า 0.05 ได้แก่ cylinders, displacement, horsepower, weight, acceleration, และ year



## 4. Variable selection

#### statsmodels library สามารถคำนวณค่าต่างๆ เหล่านี้ได้ ผลที่แสดงจะคล้ายกับใน R  
#### มีสองวิธึในการทำ linear regression: $\texttt{statsmodels.api.sm.OLS}$
#### และ $\texttt{statsmodels.formula.api.smf.ols}$

In [43]:
import statsmodels.api as sm

X = auto_df.drop(['mpg'], axis=1)
X = sm.add_constant(X)     # add bias constant
y = auto_df['mpg']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Fri, 08 Aug 2025   Prob (F-statistic):          3.79e-135
Time:                        13:12:37   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -14.5353      4.764     -3.051   

In [44]:
import statsmodels.formula.api as smf

model2 = smf.ols('mpg ~ cylinders + displacement + horsepower + weight + acceleration + year', auto_df)
model2 = model2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Fri, 08 Aug 2025   Prob (F-statistic):          3.79e-135
Time:                        13:12:39   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -14.5353      4.764     -3.051   

### Exercise 7: ใช้ Backward selection ในการเลือกตัวแปร โดยที่เราจะหยุดเมื่อ p-value สของตัวแปรทุกตัว < 0.05 แล้วเขียนสมการของโมเดลที่ได้

In [49]:

import statsmodels.formula.api as smf

model_string = 'mpg ~ cylinders + displacement + horsepower + weight + acceleration + year'
model = smf.ols(model_string, auto_df).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Fri, 08 Aug 2025   Prob (F-statistic):          3.79e-135
Time:                        13:18:36   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -14.5353      4.764     -3.051   

In [47]:
# If you want to write with for-loop

print(model2.tvalues)
print("t-value of weight:", model2.tvalues["weight"])

Intercept       -3.051136
cylinders       -0.993240
displacement     1.043586
horsepower      -0.028284
weight         -10.140877
acceleration     0.835721
year            14.317630
dtype: float64
t-value of weight: -10.140877265267768


### คำตอบของ Exercise 7:

$$
y = -3.051136 + 3.051136 * weight + 0.028284 * horsepower + 14.317630 * year
$$